In [1]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. D

In [43]:
# Example data
examples = [
    {"input": "Is it raining??", "target": "yes"},
    {"input": "Is the sky green??", "target": "no"},
    {"input": "Is the sky green??", "target": "yes"}
]

# Tokenize the inputs and targets
inputs = tokenizer([ex["input"] for ex in examples], return_tensors="pt", padding=True, truncation=True)
with tokenizer.as_target_tokenizer():
    targets = tokenizer([ex["target"] for ex in examples], return_tensors="pt", padding=True, truncation=True)

/home/wouter/miniconda3/envs/dl2023/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [44]:
inputs

{'input_ids': tensor([[ 2485,   609, 47637,   347,  1718,     1],
        [ 2485,   287, 20495, 16263,  1718,     1],
        [ 2485,   287, 20495, 16263,  1718,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [45]:
targets

{'input_ids': tensor([[36339,     1],
        [  375,     1],
        [36339,     1]]), 'attention_mask': tensor([[1, 1],
        [1, 1],
        [1, 1]])}

In [60]:
outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=targets["input_ids"])

In [61]:
outputs.logits.shape

torch.Size([3, 2, 250112])

In [62]:
outputs.logits.argmin(dim=-1), outputs.logits.max(dim=-1)

(tensor([[250028,     66],
         [250028,     66],
         [250028,     66]]),
 torch.return_types.max(
 values=tensor([[-13.6463, -16.8799],
         [-14.2549, -47.5279],
         [-14.2549, -50.2157]], grad_fn=<MaxBackward0>),
 indices=tensor([[250099, 250099],
         [250099, 250099],
         [250099, 250099]])))

In [63]:
tokenizer.decode(66), tokenizer.decode(375), tokenizer.decode(250028)

('<0x3F>', 'no', '<extra_id_71>')

In [38]:
# gebruik alleen de eerste token voor de voorspelling

In [64]:
import torch
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

In [65]:
outputs.logits[:,0,:].shape

torch.Size([3, 250112])

In [67]:
targets["input_ids"].shape

torch.Size([3, 2])

In [68]:
# aantal examples is dus B*2
loss = criterion(outputs.logits[:,0,:], targets["input_ids"][:,0])

In [69]:
loss

tensor(25.6659, grad_fn=<NllLossBackward0>)